# Getting started with pyplis

This notebook provides a step-by-step guide through a whole evaluation of pyplis and uses the [official pyplis test dataset](https://github.com/jgliss/pyplis#example-and-test-data) which is also used and documented in the [pyplis paper](https://www.mdpi.com/2076-3263/7/4/134/htm).

Essentially, what this notebook does is taking you on a tour through the [12 example scripts](https://github.com/jgliss/pyplis/tree/master/scripts) (i.e. scripts ex01_analysis_setup.py - ex12_emission_rate.py) that are discussed in the paper as well. The main steps that will be illustrated include (in chronological order):

- Define analysis setup and load dataset (classes `AnalysisSetup, Dataset`)
- Measurement geometry and plume distance retrieval (class `MeasGeometry`)

In [21]:
from warnings import filterwarnings
filterwarnings('ignore')
from datetime import datetime
import pyplis
import os

# Preface: locate example data and set basic image directory

In [22]:
DATA_DIR = pyplis.inout.find_test_data()
DATA_DIR

Found test data at location: C:\Users\Jonas\my_pyplis


'C:\\Users\\Jonas\\my_pyplis\\pyplis_etna_testdata'

The camera images can be found in sub-directory `images`:

In [23]:
IMG_DIR = os.path.join(DATA_DIR, 'images')
IMG_DIR

'C:\\Users\\Jonas\\my_pyplis\\pyplis_etna_testdata\\images'

Let's have a quick-look at what this directory contains:

In [24]:
all_files = os.listdir(IMG_DIR)
# display the first 10 files
all_files[:10]

['EC2_1106307_1R02_2015091606454457_F01_Etna.fts',
 'EC2_1106307_1R02_2015091606454717_F02_Etna.fts',
 'EC2_1106307_1R02_2015091606592799_F01_Etna.fts',
 'EC2_1106307_1R02_2015091606592966_F02_Etna.fts',
 'EC2_1106307_1R02_2015091606593268_D0L_Etna.fts',
 'EC2_1106307_1R02_2015091606593410_D1L_Etna.fts',
 'EC2_1106307_1R02_2015091606593561_D0H_Etna.fts',
 'EC2_1106307_1R02_2015091606593704_D1H_Etna.fts',
 'EC2_1106307_1R02_2015091606594030_F01_Etna.fts',
 'EC2_1106307_1R02_2015091606594215_F02_Etna.fts']

As you can see, the image files are stored using the FITS standard (.fts). The camera that was used to make these measurements is the ECII camera (NILU, Norway), which uses a file naming convention that allows to extract acquisition time from the file name (e.g. 2015091606454457 => 16.09.2015 06:45:44.457 UTC), as well as information about which filter was used for a given picture (e.g. F01, F02, D0L ...). The latter is encoded and translates to:

- F01: SO2 onband filter (~310 nm)
- F02: SO2 offband filter (~330 nm)
- D0L: Dark image, shortest exposure, low gain
- D1L: Dark image, longest exposure, low gain
- D0H: Dark image, shortest exposure, high gain
- D1H: Dark image, longest exposure, high gain

These information is actually stored in the [cam_info.txt](https://github.com/jgliss/pyplis/blob/master/pyplis/data/cam_info.txt) file that contains default information about cameras that are registered in pyplis. More information about how to setup your custom camera type can be found [here](https://pyplis.readthedocs.io/en/latest/tutorials.html#primer-on-data-import).

### Quicklook at some example images

Before diving into the analysis, let's have a quick look at one onband, offband and dark image. Images in pyplis are represented by the `Img` class and can be loaded simply by passing the file path:

In [32]:
onband_file = os.path.join(IMG_DIR, all_files[0])
offband_file = os.path.join(IMG_DIR, all_files[1])
dark_file = os.path.join(IMG_DIR, all_files[5])

In [33]:
onband = pyplis.Img(onband_file)
print(onband)

C:\Users\Jonas\my_pyplis\pyplis_etna_testdata\images\EC2_1106307_1R02_2015091606454457_F01_Etna.fts

-----------
pyplis Img
-----------

Min / Max intensity: 477.0 - 4095.0
Mean intensity: 2597.7518681117467
Shape: (1024, 1344)
ROI (abs. coords): [0, 0, 1344, 1024]

Meta information
-------------------
start_acq: 1900-01-01 00:00:00
stop_acq: 1900-01-01 00:00:00
texp: 0.0
focal_length: nan
pix_width: nan
pix_height: nan
bit_depth: nan
f_num: nan
read_gain: 0
lon: nan
lat: nan
altitude: nan
altitude_offs: nan
elev: nan
elev_err: nan
azim: nan
azim_err: nan
filter_id: 
path: C:\Users\Jonas\my_pyplis\pyplis_etna_testdata\images\EC2_1106307_1R02_2015091606454457_F01_Etna.fts
file_name: EC2_1106307_1R02_2015091606454457_F01_Etna.fts
file_type: .fts
device_id: 
ser_no: 
wvlngth: nan
fits_idx: 0
temperature: nan
user_param1: nan
user_param2: nan
user_param3: nan

Edit log
-----------
darkcorr: False
blurring: 0
median: 0
crop: False
8bit: False
pyrlevel: 0
is_tau: False
is_aa: False
vigncorr:

The print command of the image class yields useful output. As you can see, most meta information is unspecified. If your camera is registered in pyplis and specs are available in file *cam_info.txt* (which is the case here ofc), you may use your camera ID to register further camera specific information when loading an image: 

In [35]:
offband = pyplis.Img(offband_file, cam_id='ecII')
print(offband)

C:\Users\Jonas\my_pyplis\pyplis_etna_testdata\images\EC2_1106307_1R02_2015091606454717_F02_Etna.fts

-----------
pyplis Img
-----------

Min / Max intensity: 530.0 - 3444.0
Mean intensity: 2495.688948858352
Shape: (1024, 1344)
ROI (abs. coords): [0, 0, 1344, 1024]

Meta information
-------------------
start_acq: 1900-01-01 00:00:00
stop_acq: 1900-01-01 00:00:00
texp: 0.0
focal_length: nan
pix_width: nan
pix_height: nan
bit_depth: nan
f_num: nan
read_gain: 0
lon: nan
lat: nan
altitude: nan
altitude_offs: nan
elev: nan
elev_err: nan
azim: nan
azim_err: nan
filter_id: 
path: C:\Users\Jonas\my_pyplis\pyplis_etna_testdata\images\EC2_1106307_1R02_2015091606454717_F02_Etna.fts
file_name: EC2_1106307_1R02_2015091606454717_F02_Etna.fts
file_type: .fts
device_id: 
ser_no: 
wvlngth: nan
fits_idx: 0
temperature: nan
user_param1: nan
user_param2: nan
user_param3: nan

Edit log
-----------
darkcorr: False
blurring: 0
median: 0
crop: False
8bit: False
pyrlevel: 0
is_tau: False
is_aa: False
vigncorr: 

In [ ]:
onband.show(vmin=0, vmax=2**12); #use ; in last cell entry to surpress output

## Define analysis setup and load dataset

In the following, relevant parameter for the analysis are defined. From these, an instance of the class `AnalysisSetup` is created, which is used to create an instance of the `Dataset` class.

Define start and stop time of the measurement

In [15]:
start_plume = datetime(2015, 9, 16, 7, 6, 00)
stop_plume = datetime(2015, 9, 16, 7, 22, 00)

# Define camera (here the default ecII type is used)
cam_id = "ecII"

# the camera filter setup
filters = [pyplis.utils.Filter(type="on", acronym="F01"),
           pyplis.utils.Filter(type="off", acronym="F02")]

# camera location and viewing direction (altitude will be retrieved
# automatically)
geom_cam = {"lon": 15.1129,
            "lat": 37.73122,
            "elev": 20.0,
            "elev_err": 5.0,
            "azim": 270.0,
            "azim_err": 10.0,
            "alt_offset": 15.0,
            "focal_length": 25e-3}  # altitude offset (above topography)

# Create camera setup
# the camera setup includes information about the filename convention in
# order to identify different image types (e.g. on band, off band, dark..)
# it furthermore includes information about the detector specifics (e.g.
# dimension, pixel size, focal length). Measurement specific parameters
# (e.g. lon, lat, elev, azim) where defined in the dictinary above and
# can be passed as additional keyword dictionary using **geom_cam
# Alternatively, they could also be passed directly, e.g.:

# cam = pyplis.setup.Camera(cam_id, filter_list=filters, lon=15.1129,
#                           lat=37.73122)

cam = pyplis.setupclasses.Camera(cam_id, filter_list=filters,
                                 **geom_cam)

# Load default information for Etna. This information is stored in
# the source_info.txt file of the Pyplis information. You may also access
# information about any volcano via the available online access to the NOAA
# database using the method pyplis.inout.get_source_info_online(source_id).

source = pyplis.setupclasses.Source("etna")

# Provide wind direction
wind_info = {"dir": 0.0,
             "dir_err": 1.0}

#                "dir_err"  : 15.0}

# Create BaseSetup object (which creates the MeasGeometry object)
stp = pyplis.setupclasses.MeasSetup(IMG_DIR, start, stop, camera=cam,
                                    source=source,
                                    wind_info=wind_info)
print(stp.LINK_OFF_TO_ON)
# Create analysis object (from BaseSetup)
# The dataset takes care of finding all vali
ds = pyplis.Dataset(stp)

True
Retrieving SRTM data (this might take a while) ... 
Updated camera in GeoSetup of MeasGeometry
Updated source in GeoSetup of MeasGeometry
Updated source2cam GeoVector in GeoSetup of MeasGeometry
Updated camera CFOV vector in GeoSetup of MeasGeometry
Updated plume vector in GeoSetup of MeasGeometry
Updated GeoPoint of intersection between camera CFOV and plume vector in GeoSetup of MeasGeometry
MeasGeometry was updated and fulfills all requirements
True
Updating measurement setup in Dataset

SEARCHING VALID FILE PATHS IN
C:\Users\Jonas\my_pyplis\pyplis_etna_testdata\images

Image search is only performed in specified directory and does not include subdirectories
Using only fts files
Total number of files found 509
INFO: texp cannot be accessed from filename
422 files of type were found in specified time interval 2015-09-16 07:06:00 - 2015-09-16 07:22:00
422 files of type were found in specified time interval 2015-09-16 07:06:00 - 2015-09-16 07:22:00

Init ImgList on
---------------